In [79]:
import pathlib
import json
import os
import math
import random
import numpy as np
import cv2
from PIL import Image, ExifTags, ImageEnhance, ImageOps
from concurrent.futures import ThreadPoolExecutor, as_completed

In [80]:
def get_average_color(img):
    average_color = np.average(np.average(img, axis=0), axis=0)
    average_color = np.around(average_color, decimals=-1)
    average_color = tuple(int(i) for i in average_color)
    return average_color
  
def get_closest_color(color, colors):
    cr, cg, cb = color

    min_difference = float("inf")
    closest_color = None
    for c in colors:
        r, g, b = eval(c)
        difference = math.sqrt((r - cr) ** 2 + (g - cg) ** 2 + (b - cb) ** 2)
        if difference < min_difference:
            min_difference = difference
            closest_color = eval(c)

    return closest_color

def crop_to_square_center(image_path, output_path_base, filter_settings_file='settings.json'):

    filter_settings = read_settings(filter_settings_file)
    
    with Image.open(image_path) as img:
        
        img = correct_orientation(img)
        
        width, height = img.size
        
        new_size = min(width, height)
        
        # Calculate the coordinates to create the new image
        left = (width - new_size)/2
        top = (height - new_size)/2
        right = (width + new_size)/2
        bottom = (height + new_size)/2
        
        img_cropped = img.crop((left, top, right, bottom))
        
        # Resize for tile folder based on settings
        stored_tile_size = filter_settings["stored_tile_size_in_px"]
        newsize = (stored_tile_size, stored_tile_size)
        img_cropped = img_cropped.resize(newsize)
        
        # Save tiles for filters specified in settings
        new_path = change_directory(image_path, output_path_base)
        
        if filter_settings["include_no_filter"]:
            img_cropped.save(new_path)
        
        if filter_settings["apply_grayscale_filter"]:
            gray_img = img_cropped.convert("L")
            gray_img.save(f"{new_path}_grayscale.jpg")

        if filter_settings["apply_sepia_filter"]:
            sepia_img = apply_sepia(img_cropped)
            sepia_img.save(f"{new_path}_sepia.jpg")

        if filter_settings["apply_cool_filter"]:
            cool_img = apply_cool_filter(img_cropped)
            cool_img.save(f"{new_path}_cool.jpg")

        if filter_settings["apply_vintage_filter"]:
            vintage_img = apply_vintage_filter(img_cropped)
            vintage_img.save(f"{new_path}_vintage.jpg")
        
        if filter_settings["apply_red_filter"]:
            red_img = apply_red_filter(img_cropped)
            red_img.save(f"{new_path}_red.jpg")
        
        if filter_settings["apply_green_filter"]:
            green_img = apply_green_filter(img_cropped)
            green_img.save(f"{new_path}_green.jpg")
        
        if filter_settings["apply_blue_filter"]:
            blue_img = apply_blue_filter(img_cropped)
            blue_img.save(f"{new_path}_blue.jpg")

def correct_orientation(img):
    try:
        for orientation in ExifTags.TAGS.keys():
            if ExifTags.TAGS[orientation] == 'Orientation':
                break
        exif = dict(img._getexif().items())

        if exif[orientation] == 3:
            img = img.rotate(180, expand=True)
        elif exif[orientation] == 6:
            img = img.rotate(270, expand=True)
        elif exif[orientation] == 8:
            img = img.rotate(90, expand=True)
    except (AttributeError, KeyError, IndexError):
        # cases: image don't have getexif
        pass
    return img

def change_directory(file_path, new_directory):
    # Split the path into head and tail
    head, tail = os.path.split(file_path)
    
    # Replace 'input_images' with 'tiles'
    new_head = new_directory
    
    # Combine the new head and the original tail
    new_path = os.path.join(new_head, tail)
    
    return new_path

def apply_cool_filter(img):
    # Convert to grayscale
    gray_img = img.convert('L')
    # Apply a blue tint
    blue_img = Image.new('RGB', img.size, (0, 70, 255))
    return Image.blend(gray_img.convert('RGB'), blue_img, 0.1)

def apply_vintage_filter(img):
    # Apply a warm-tone effect
    red_img = Image.new('RGB', img.size, (255, 70, 0))
    return Image.blend(img, red_img, 0.1)

def apply_sepia(img):
    # Convert the image to grayscale
    grayscale_img = img.convert('L')
    # Applying a sepia tone requires adding red and green to the image, here's a simple sepia tone
    sepia_img = Image.new("RGB", img.size, (255, 240, 192))
    # Blend the sepia image with the grayscale image
    return Image.blend(sepia_img, grayscale_img.convert("RGB"), 0.5)

# Helper function to create a color ramp
def make_linear_ramp(base_color):
    ramp = []
    for i in range(255):
        ramp.extend((int(base_color[0] * i / 255), int(base_color[1] * i / 255), int(base_color[2] * i / 255)))
    return ramp

def apply_red_filter(img):
    # Create a solid red image
    red_img = Image.new('RGB', img.size, (255, 0, 0))
    # Blend the grayscale image with the red image
    return Image.blend(img.convert('RGB'), red_img, 0.5)

def apply_green_filter(img):
    # Create a solid green image
    green_img = Image.new('RGB', img.size, (0, 255, 0))
    # Blend the grayscale image with the green image
    return Image.blend(img.convert('RGB'), green_img, 0.5)

def apply_blue_filter(img):
    # Create a solid blue image
    blue_img = Image.new('RGB', img.size, (0, 0, 255))
    # Blend the grayscale image with the blue image
    return Image.blend(img.convert('RGB'), blue_img, 0.5)

# Separate function to process a single tile
def process_tile(tile, img, data, tile_width, tile_height):
    y0, y1, x0, x1 = tile
    
    # Check if the tile dimensions are valid
    if y1 <= y0 or x1 <= x0:
        print(f"Invalid tile dimensions: {tile}")
        return tile, None

    try:
        # Extract the part of the image that corresponds to the current tile
        tile_img = img[y0:y1, x0:x1].copy()

        # Ensure there are pixels in the tile before processing
        if tile_img.size == 0:
            print(f"Empty tile: {tile}")
            return tile, None

        # Get the average color and closest color for the current tile
        average_color = get_average_color(tile_img)
        
        # Ensure that average_color is valid
        if average_color is None:
            print(f"Could not determine average color for tile {tile}")
            return tile, None

        closest_color = get_closest_color(average_color, data.keys())

        # Ensure that a closest color is found
        if closest_color is None:
            print(f"No closest color found for average color {average_color}")
            return tile, None

        # Choose a random image path from the data and read the image
        i_path = random.choice(data[str(closest_color)])
        i = cv2.imread(i_path)
        
        # Ensure the image is read correctly
        if i is None:
            print(f"Failed to read image from path: {i_path}")
            return tile, None

        # Resize the image to fit the tile size
        i_resized = cv2.resize(i, (tile_width, tile_height))

        # Replace the content of the tile with the new image
        img[y0:y1, x0:x1] = i_resized

    except Exception as e:
        print(f"Error processing tile {tile}: {e}")
        return tile, None

    return tile, img[y0:y1, x0:x1]

# Function to initialize default settings
def initialize_default_settings():
    return {
        "include_no_filter": True,
        "apply_grayscale_filter": False,
        "apply_sepia_filter": False,
        "apply_cool_filter": False,
        "apply_vintage_filter": False,
        "apply_red_filter": False,
        "apply_green_filter": False,
        "apply_blue_filter": False,
        "stored_tile_size_in_px" : 50
    }

# Function to write settings to a JSON file
# Example usage:
# settings_to_write = {"apply_sepia_filter": True}
# write_settings(settings_to_write)
def write_settings(settings, file_path='settings.json'):
    default_settings = initialize_default_settings()
    # Update default settings with any provided settings
    default_settings.update(settings)

    with open(file_path, 'w') as file:
        json.dump(default_settings, file, indent=4)

# Function to read settings from a JSON file
def read_settings(file_path='settings.json'):
    default_settings = initialize_default_settings()

    try:
        with open(file_path, 'r') as file:
            file_settings = json.load(file)
            default_settings.update(file_settings)
    except FileNotFoundError:
        pass

    return default_settings

# Set of functions to confirm image list has not changed
def get_input_image_list(folder_path):
    file_list = []
    
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            file_list.append(file)
    
    return file_list

def save_input_image_list_to_json(file_list, file_check_json):
    if file_check_json not in os.listdir():
        with open(file_check_json, "w") as new_check_file:
            json.dump(file_list, new_check_file, indent=2, sort_keys=True)

def get_cached_file_list(file_path):
    with open(file_path, "r") as f:
        return json.load(f)

def get_input_image_path_list(folder_path):
    file_list = []
  
    # Supported image file extensions
    extensions = ["*.jpg"]
        
    for extension in extensions:
        file_list.extend(folder_path.rglob(extension))
  
    return [str(file) for file in file_list]
    

In [81]:
# Mosaic Settings
tile_height_in_px = 50 # Creating the mosaic with smaller tiles effectively "increases" the resolution of the whole image, but it will be harder to see the individual tiles
output_image_height_in_px = 1500 # Increasing the output image size might be important if printing size is large
stored_tile_size_in_px = 50 # Storing smaller tiles will reduce the harddrive space required, but this will be the maximum resolution for each tile

# Tile filter settings - including different filters might give you more variations in which tiles are chosen, but will increase required harddrive space
tile_settings_to_write = {"include_no_filter": True,
                          "apply_grayscale_filter": False,
                          "apply_sepia_filter": False,
                          "apply_cool_filter": False,
                          "apply_vintage_filter": False,
                          "apply_red_filter": False,
                          "apply_green_filter": False,
                          "apply_blue_filter": False,
                          "stored_tile_size_in_px": stored_tile_size_in_px}

# Folders
input_images_folder_name = 'input_images'
tile_images_folder_name = 'tiles'

# Files
output_target_image_name = 'output_target_image.jpg'
output_image_name = 'output.jpg'
tile_filter_settings_file = 'settings.json'
file_check_json = 'filecheck.json'
tile_avg_color_cache_json = 'cache.json'

In [82]:
tile_settings_changed = False
tile_settings = read_settings(tile_filter_settings_file)

if tile_settings_to_write != tile_settings:
  tile_settings_changed = True
  write_settings(tile_settings_to_write)

In [83]:
input_imgs_dir = pathlib.Path(input_images_folder_name)

current_file_list = get_input_image_list(input_imgs_dir)

# Check if there are images in the input folder
if len(current_file_list) == 0:
  print(f"Error - no images in {input_imgs_dir}, or destination folder does not exist.")

In [84]:
file_list_updated = True

# Check if input images have been added or removed
if pathlib.Path(file_check_json).exists():
  prior_file_list = get_cached_file_list(file_check_json)
  if current_file_list == prior_file_list:
    file_list_updated = False
  else:
    save_input_image_list_to_json(current_file_list, file_check_json)
else:
  save_input_image_list_to_json(current_file_list, file_check_json)

In [85]:
# Check if the tile list needs an update (if file list changed, if tile folder is empty, or if tile settings changed)
tile_list_empty = False
existing_tile_height_in_px = 0
tile_imgs_dir = pathlib.Path(tile_images_folder_name)
input_image_path_list = get_input_image_path_list(input_imgs_dir)

if len(os.listdir(tile_imgs_dir)) == 0:
  tile_list_empty = True
  
# if not tile_list_empty:
#   first_image_path = os.path.join(tile_imgs_dir, os.listdir(tile_imgs_dir)[0])
#   with Image.open(first_image_path) as img:
#     existing_tile_height_in_px = img.height
#     tile_settings_changed = stored_tile_size_in_px != existing_tile_height_in_px

tiles_need_update = file_list_updated or tile_list_empty or tile_settings_changed


In [86]:
tile_list_updated = False

if tiles_need_update:
  print("Tile list is empty or input files were updated since last run. Re-creating tiles. This may take a while, depending on the number of input images and filter passes requested.")
  tile_images = list(tile_imgs_dir.glob("*.jpg"))
  
  # Delete old tiles first
  for old_tile in tile_images:
    os.remove(old_tile)
  
  # Create new tiles
  for img_path in input_image_path_list:
    crop_to_square_center(img_path, tile_imgs_dir, tile_filter_settings_file)
  tile_list_updated = True
  print("Tiles created.")

Tile list is empty or input files were updated since last run. Re-creating tiles. This may take a while, depending on the number of input images and filter passes requested.
Tiles created.


In [87]:
if not pathlib.Path(tile_avg_color_cache_json).exists() or tile_list_updated:
    print("Caching average color for each tile. This may take a while.")
    tile_images = list(tile_imgs_dir.glob("*.jpg"))
    data = {}
    
    for img_path in tile_images:
        img = cv2.imread(str(img_path))
        average_color = get_average_color(img)
        if str(tuple(average_color)) in data:
            data[str(tuple(average_color))].append(str(img_path))
        else:
            data[str(tuple(average_color))] = [str(img_path)]
    with open(tile_avg_color_cache_json, "w") as file:
        json.dump(data, file, indent=2, sort_keys=True)
    print("Average colors cached.")

Caching average color for each tile. This may take a while.
Average colors cached.


In [88]:
# Load average colors for each tile
with open(tile_avg_color_cache_json, "r") as file:
    data = json.load(file)

# Read target image for mosaic
if pathlib.Path(output_target_image_name).exists():
    img = cv2.imread(output_target_image_name)
else:
    print(f"Error - {output_target_image_name} does not exist")

In [89]:
# Set final image size
img_height_orig, img_width_orig, _ = img.shape
output_image_width_in_px = round(img_width_orig / img_height_orig * output_image_height_in_px)

img_resized = cv2.resize(img, (output_image_width_in_px, output_image_height_in_px))

# Set tile sizes
tile_height, tile_width = tile_height_in_px, tile_height_in_px
num_tiles_h, num_tiles_w = output_image_height_in_px // tile_height, output_image_width_in_px // tile_width
img_resized = img_resized[:tile_height * num_tiles_h, :tile_width * num_tiles_w]

In [90]:
tiles = []
for y in range(0, output_image_height_in_px, tile_height):
    for x in range(0, output_image_width_in_px, tile_width):
        tiles.append((y, y + tile_height, x, x + tile_width))

In [91]:
print("Creating image. This may take a while, depending on how large the output image needs to be (bigger images take more time) and how small the tiles need to be (smaller tiles take more time, as more are needed).")
# Prepare arguments for each tile processing
args = [(tile, img_resized, data, tile_width, tile_height) for tile in tiles]

# Use ThreadPoolExecutor to process tiles in parallel
results = {}
with ThreadPoolExecutor(max_workers=4) as executor:
    future_to_tile = {executor.submit(process_tile, *arg): arg[0] for arg in args}
    for future in as_completed(future_to_tile):
        tile, result = future.result()
        if result is not None:
            y0, y1, x0, x1 = tile
            img_resized[y0:y1, x0:x1] = result

# Write the final image to a file
cv2.imwrite(output_image_name, img_resized)
print(f"{output_image_name} created.")

Creating image. This may take a while, depending on how large the output image needs to be (bigger images take more time) and how small the tiles need to be (smaller tiles take more time, as more are needed).
output.jpg created.
